In [1]:
import pandas as pd
pd.options.display.max_columns = None

In [ ]:
#from google.colab import drive
#drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [2]:
!pip install clickhouse-driver -q

     |████████████████████████████████| 608 kB 25.3 MB/s 


In [3]:
#import pandas as pd
import sys
from clickhouse_driver import Client

#if not 'pd' in globals():
#    pd = None

default_type_mapping = {'object': 'String', 'int64': 'UInt64', 'float64': 'Float64', 'datetime64[ns]': 'DateTime'}

class DfClient(Client):
    def __init__(self, *args, **kwargs):
        self.df_engine = kwargs.pop('df_engine', None)
        if self.df_engine is None:
            raise Exception("df_engine wasn't specified in constructor arguments")#. Tried to use pd, but it isn't imported")
        super().__init__(*args, **kwargs)
    
    def columns(self, tablename):
        result = self.execute('DESCRIBE TABLE ' + tablename)
        columnlist = list(map(lambda x: x[0], result))
        return columnlist

    def query(self, columns, query):
        result = self.execute(query)
        return df_engine.DataFrame(result, columns=columns)

    def select(self, tablename, columns, where_and_after=''):
        columns_string = ', '.join(columns)
        return self.query(columns, f'SELECT {columns_string} FROM {tablename} {where_and_after}')

    def select_all_columns(self, tablename, where_and_after=''):
        # Просто * не подойдет, так как не включает столбцы Materialized и Alias
        columns = self.columns(tablename)
        return self.select(tablename, columns, where_and_after)

    def insert(self, tablename, df):
        tuples = df.to_dict(orient='records')
        #tuples = df.itertuples(index=False, name=None)
        self.execute(f'INSERT INTO {tablename} VALUES', tuples)

    def insert_to_new_table(self, tablename, df, order_key=None, engine='MergeTree()', type_mapping=default_type_mapping):
        if order_key is None:
            order_key = df.columns[0]
        query = f'CREATE TABLE {tablename} ('
        for column, dtype in df.dtypes.iteritems():
            query += f'{column} {type_mapping[str(dtype)]},'
        query = query[:-1] # Удаление последнего символа (лишней запятой)
        query += f') ENGINE = {engine} ORDER BY {order_key}'
        self.execute(query)
        self.insert(tablename, df)

    def drop(self, tablename):
        self.execute('DROP TABLE ' + tablename)

    def truncate(self, tablename):
        self.execute('TRUNCATE TABLE ' + tablename)
    
    def create_column(self, tablename, column_name, default_expression, column_type='ALIAS'):
        self.execute(f'ALTER TABLE {tablename} ADD COLUMN {column_name} {column_type} {default_expression}')

In [5]:
#import sys
#sys.path.append("/content/drive/MyDrive/Интеллектуальный анализ данных в бизнесе 2 Гурьянов 11-909/Задача 2")
#from dfclient import DfClient

client = DfClient('7.tcp.eu.ngrok.io', port='18315', user='default', password='default', database='default', df_engine=pd)

In [6]:
import random
from datetime import datetime
from sklearn import datasets
import pandas as pd

iris_data = datasets.load_iris()
df_iris = pd.DataFrame(iris_data.data,columns=iris_data.feature_names)
df_iris['target'] = pd.Series(iris_data.target)
df_iris['string_column'] = df_iris.apply(lambda x: "qwerty" + str(random.random()), axis=1)
df_iris['date_column'] = df_iris.apply(lambda x: datetime.now(), axis=1)
df_iris.rename({'sepal length (cm)': 'sepal_length', 'sepal width (cm)': 'sepal_width', 'petal length (cm)': 'petal_length', 'petal width (cm)': 'petal_width'}, axis=1, inplace=True)
df_iris

,sepal_length,sepal_width,petal_length,petal_width,target,string_column,date_column
0,5.1,3.5,1.4,0.2,0,qwerty0.912660679146642,2022-05-13 07:06:32.859918
1,4.9,3.0,1.4,0.2,0,qwerty0.3034405619694087,2022-05-13 07:06:32.859934
2,4.7,3.2,1.3,0.2,0,qwerty0.44341094413449167,2022-05-13 07:06:32.859940
3,4.6,3.1,1.5,0.2,0,qwerty0.2342236487302487,2022-05-13 07:06:32.859946
4,5.0,3.6,1.4,0.2,0,qwerty0.47199023804893114,2022-05-13 07:06:32.859951
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,qwerty0.07480947347752154,2022-05-13 07:06:32.860736
146,6.3,2.5,5.0,1.9,2,qwerty0.9054951469970415,2022-05-13 07:06:32.860741
147,6.5,3.0,5.2,2.0,2,qwerty0.40686268219706745,2022-05-13 07:06:32.860747
148,6.2,3.4,5.4,2.3,2,qwerty0.05064929563684073,2022-05-13 07:06:32.860752


In [7]:
client.insert_to_new_table('iris', df_iris)

Failed to connect to 7.tcp.eu.ngrok.io:18315
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/clickhouse_driver/connection.py", line 325, in connect
    return self._init_connection(host, port)
  File "/usr/local/lib/python3.7/dist-packages/clickhouse_driver/connection.py", line 289, in _init_connection
    self.socket = self._create_socket(host, port)
  File "/usr/local/lib/python3.7/dist-packages/clickhouse_driver/connection.py", line 261, in _create_socket
    raise err
  File "/usr/local/lib/python3.7/dist-packages/clickhouse_driver/connection.py", line 252, in _create_socket
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused


NetworkError: ignored

In [ ]:
result = client.execute('SELECT * FROM iris LIMIT 5')
pd.DataFrame(result)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,2.0,1,88,10,52680,170,573,905,2020-09-16,173,4,3.0,Missing,Missing,Missing,АВТОБУС (городской маршрут),№ транспортного средства 152,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing
1,Missing,1,19,13,18280,217,2245,675,2020-09-16,213,1,Missing,Missing,Missing,Missing,Missing,Missing,Missing,2.0,-1.0,Missing,Missing,3.0,Missing,Missing
2,Missing,1,665,18,23046,233,2535,921,2020-09-14,228,1,3.0,Missing,noreplyofd@kontur.ru,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing
3,2.0,1,356,28,136770,339,1727,689,2020-09-16,76,1,3.0,Missing,Missing,Missing,Missing,Missing,Missing,2.0,Missing,Missing,Missing,Missing,Missing,Missing
4,Missing,1,10,37,124784,459,778,1228,2020-09-14,2370,1,3.0,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing


In [ ]:
#def column_list(client, tablename):
#    result = client.execute('DESCRIBE TABLE ' + tablename)
#    columnlist = list(map(lambda x: x[0], result))
#    return columnlist
columns = client.columns('iris')
columns

['protocolVersion',
 'operationType',
 'shiftNumber',
 'totalSum',
 'ecashTotalSum',
 'nds10',
 'nds18',
 'nds20',
 'dateTime',
 'requestNumber',
 'taxationType',
 'receiptCode',
 'internetSign',
 'senderAddress',
 'buyerAddress',
 'userProperty_key',
 'userProperty_value',
 'sellerAddress',
 'fiscalDocumentFormatVer',
 'paymentAgentType',
 'propertiesUser_propertyName',
 'retailAddress',
 'code',
 'propertiesUser_key',
 'propertiesUser_value']

In [ ]:
#def select_df(client, columns, query):
#    result = client.execute(query)
#    return pd.DataFrame(result, columns=columns)

#def select_df(client, tablename, condition, columns=None):
#    if columns is None:
#        columns = column_list(client, tablename)
#    result = client.execute(f'SELECT *')
#    return pd.DataFrame(result, columns=columns)

#select_df(client, columns, 'SELECT * FROM fns_fiscal LIMIT 5')

client.query(columns, 'SELECT * FROM iris LIMIT 5')

,protocolVersion,operationType,shiftNumber,totalSum,ecashTotalSum,nds10,nds18,nds20,dateTime,requestNumber,taxationType,receiptCode,internetSign,senderAddress,buyerAddress,userProperty_key,userProperty_value,sellerAddress,fiscalDocumentFormatVer,paymentAgentType,propertiesUser_propertyName,retailAddress,code,propertiesUser_key,propertiesUser_value
0,2.0,1,88,10,52680,170,573,905,2020-09-16,173,4,3.0,Missing,Missing,Missing,АВТОБУС (городской маршрут),№ транспортного средства 152,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing
1,Missing,1,19,13,18280,217,2245,675,2020-09-16,213,1,Missing,Missing,Missing,Missing,Missing,Missing,Missing,2.0,-1.0,Missing,Missing,3.0,Missing,Missing
2,Missing,1,665,18,23046,233,2535,921,2020-09-14,228,1,3.0,Missing,noreplyofd@kontur.ru,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing
3,2.0,1,356,28,136770,339,1727,689,2020-09-16,76,1,3.0,Missing,Missing,Missing,Missing,Missing,Missing,2.0,Missing,Missing,Missing,Missing,Missing,Missing
4,Missing,1,10,37,124784,459,778,1228,2020-09-14,2370,1,3.0,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing


In [ ]:
#def insert_df(client, tablename, df):
#    tuples = df.to_dict(orient='records')
    #tuples = df.itertuples(index=False, name=None)
 #   result = client.execute(f'INSERT INTO {tablename} VALUES', tuples)

In [ ]:
#df1 = select_df(client, columns, 'SELECT * FROM fns_fiscal WHERE totalSum = 196891')
df1 = client.select_all_columns('iris', 'WHERE petal_length = 6.9')
df1

,protocolVersion,operationType,shiftNumber,totalSum,ecashTotalSum,nds10,nds18,nds20,dateTime,requestNumber,taxationType,receiptCode,internetSign,senderAddress,buyerAddress,userProperty_key,userProperty_value,sellerAddress,fiscalDocumentFormatVer,paymentAgentType,propertiesUser_propertyName,retailAddress,code,propertiesUser_key,propertiesUser_value
0,Missing,1,191,196891,133039,67,3477,253,2020-09-14,46,4,3.0,Missing,Missing,Missing,АВТОБУС (городской маршрут),№ транспортного средства 152,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing
1,Missing,1,191,196891,133039,67,3477,253,2020-09-14,46,4,3.0,Missing,Missing,Missing,АВТОБУС (городской маршрут),№ транспортного средства 152,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing
2,Missing,1,191,196891,133039,67,3477,253,2020-09-14,46,4,3.0,Missing,Missing,Missing,АВТОБУС (городской маршрут),№ транспортного средства 152,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing
3,Missing,1,191,196891,133039,67,3477,253,2020-09-14,46,4,3.0,Missing,Missing,Missing,АВТОБУС (городской маршрут),№ транспортного средства 152,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing
4,Missing,1,191,196891,133039,67,3477,253,2020-09-14,46,4,3.0,Missing,Missing,Missing,АВТОБУС (городской маршрут),№ транспортного средства 152,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing
5,Missing,1,191,196891,133039,67,3477,253,2020-09-14,46,4,3.0,Missing,Missing,Missing,АВТОБУС (городской маршрут),№ транспортного средства 152,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing
6,Missing,1,191,196891,133039,67,3477,253,2020-09-14,46,4,3.0,Missing,Missing,Missing,АВТОБУС (городской маршрут),№ транспортного средства 152,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing
7,Missing,1,191,196891,133039,67,3477,253,2020-09-14,46,4,3.0,Missing,Missing,Missing,АВТОБУС (городской маршрут),№ транспортного средства 152,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing


In [ ]:
client.insert('iris', df1)

In [ ]:
#select_df(client, columns, 'SELECT * FROM fns_fiscal WHERE totalSum = 196891')
client.select('iris', ['petal_length'], 'WHERE totalSum = 196891')

,totalSum
0,196891
1,196891
2,196891
3,196891
4,196891
5,196891
6,196891
7,196891
8,196891
9,196891


,sepal_length,sepal_width,petal_length,petal_width,target,string_column,date_column
0,5.1,3.5,1.4,0.2,0,qwerty0.5473512480711592,2022-04-08 16:22:21.638460
1,4.9,3.0,1.4,0.2,0,qwerty0.40955619831773793,2022-04-08 16:22:21.638476
2,4.7,3.2,1.3,0.2,0,qwerty0.32981910679105486,2022-04-08 16:22:21.638482
3,4.6,3.1,1.5,0.2,0,qwerty0.41312523107747423,2022-04-08 16:22:21.638488
4,5.0,3.6,1.4,0.2,0,qwerty0.8708038916973392,2022-04-08 16:22:21.638493
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,qwerty0.3049709463935849,2022-04-08 16:22:21.639258
146,6.3,2.5,5.0,1.9,2,qwerty0.1588577424556713,2022-04-08 16:22:21.639263
147,6.5,3.0,5.2,2.0,2,qwerty0.9710001815106231,2022-04-08 16:22:21.639268
148,6.2,3.4,5.4,2.3,2,qwerty0.25622334143259806,2022-04-08 16:22:21.639273


In [ ]:
#df_iris.dtypes

In [ ]:
#client.execute('DROP TABLE iris')

In [ ]:
#def insert_df_to_new_table(client, tablename, df, order_key=None, engine='MergeTree()'):
#    type_mapping = {'object': 'String', 'int64': 'UInt64', 'float64': 'Float64', 'datetime64[ns]': 'DateTime'}
#    if order_key is None:
#        order_key = df.columns[0]
#    query = f'CREATE TABLE {tablename} ('
#    for column, dtype in df.dtypes.iteritems():
#        query += f'{column} {type_mapping[str(dtype)]},'
#    query = query[:-1] # Удаление последнего символа (лишней запятой)
#    query += f') ENGINE = {engine} ORDER BY {order_key}'
#    client.execute(query)
#    insert_df(client, tablename, df)

In [ ]:
#select_df(client, list(df_iris.columns), 'SELECT * FROM iris')
client.select_all_columns('iris')

,sepal_length,sepal_width,petal_length,petal_width,target,string_column,date_column
0,4.3,3.0,1.1,0.1,0,qwerty0.44217652438180943,2022-04-08 16:22:21
1,4.4,2.9,1.4,0.2,0,qwerty0.6454904030318409,2022-04-08 16:22:21
2,4.4,3.0,1.3,0.2,0,qwerty0.7660666159408859,2022-04-08 16:22:21
3,4.4,3.2,1.3,0.2,0,qwerty0.3451000084500814,2022-04-08 16:22:21
4,4.5,2.3,1.3,0.3,0,qwerty0.24219333784735286,2022-04-08 16:22:21
...,...,...,...,...,...,...,...
145,7.7,3.8,6.7,2.2,2,qwerty0.8134610740625478,2022-04-08 16:22:21
146,7.7,2.6,6.9,2.3,2,qwerty0.4140368909744152,2022-04-08 16:22:21
147,7.7,2.8,6.7,2.0,2,qwerty0.8678667793024423,2022-04-08 16:22:21
148,7.7,3.0,6.1,2.3,2,qwerty0.2662340483992097,2022-04-08 16:22:21


In [ ]:
client.truncate('iris')
client.select_all_columns('iris')

,sepal_length,sepal_width,petal_length,petal_width,target,string_column,date_column


In [ ]:
client.drop('iris')

In [ ]:
#list(select_df(client, columns, 'SELECT * FROM fns_fiscal LIMIT 5').itertuples(index=False, name=None))

In [ ]:
#import pandas as pd

#data = pd.read_csv('/content/drive/MyDrive/Интеллектуальный анализ данных в бизнесе 2 Гурьянов 11-909/Задача 2/fns_fiscal_generated_100k.csv')
#data.head()

In [ ]:
#!pip install pandahouse

In [ ]:
#from pandahouse import to_clickhouse
#client = Client('rc1b-pntrmzcmzwb36wdc.mdb.yandexcloud.net', user='user1', password='11111111', database='metrica_data', secure=True)
#connection = {'host': 'https://rc1b-pntrmzcmzwb36wdc.mdb.yandexcloud.net:9440',
#              'database': 'metrica_data',
#              'user': 'user1',
#              'password': '11111111',
#              'secure': True,
#              'port': 9440,
#              'ssl': True}
#affected_rows = to_clickhouse(data, table='fns', connection=connection, verify=False)
#affected_rows

In [ ]:
#import importlib
#import dfclient
#importlib.reload(dfclient)

<module 'dfclient' from '/content/drive/MyDrive/Интеллектуальный анализ данных в бизнесе 2 Гурьянов 11-909/Задача 2/dfclient.py'>